In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from numpy import linalg as la
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from dppy.finite_dpps import FiniteDPP
import seaborn as sns
import scipy.io as sio
import time
import matplotlib.pyplot as plt 


# To plot consistent and pretty figures
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rcParams['font.family'] = 'times new roman'
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.dpi'] = 120
mpl.rcParams['savefig.dpi'] = 120
import matplotlib
matplotlib.axes.Axes.contour
matplotlib.pyplot.contour
matplotlib.axes.Axes.contourf
matplotlib.pyplot.contourf
matplotlib.figure.Figure.colorbar
matplotlib.pyplot.colorbar
matplotlib.axes.Axes.legend
matplotlib.pyplot.legend
matplotlib.contour.ContourSet
matplotlib.contour.ContourSet.legend_elements
def landmark_uniform(X, m):
    '''Uniform landmark selection'''
    # Select m landmarks uniformly at random from the data set X
    n = len(X)  # Total number of data points
    ind = np.random.choice(n, m, replace=False)  # Randomly choose m indices without replacement
    return X[ind]  # Return the selected landmarks

def landmark_kmeans(X, m):
    '''K-means landmark selection'''
    # Apply K-means clustering to the data set X and select the m cluster centroids as landmarks
    return KMeans(n_clusters=m, max_iter=100).fit(X).cluster_centers_

def landmark_dpp(K_org, X, max_eig, m):
    '''DPP landmark selection'''
    # Use the Determinantal Point Process (DPP) to select m landmarks based on the correlation kernel
    DPP = FiniteDPP(kernel_type='correlation', projection=False, **{'K': K_org / max_eig})
    try:
        ind = DPP.sample_mcmc_k_dpp(size=m)  # Sample m landmarks using MCMC-based DPP sampling
        return X[ind]  # Return the selected landmarks
    except:
        return np.ones((m, X.shape[1]))  # If an error occurs, return a matrix of ones (fallback option)

def landmark_importanceSampling(X, sigma, m, frac, omega):
    '''Proposed importance sampling landmark selection
       Coarse-to-fine landmark selection strategy:
       m/2: uniform
       m/2: based on the importance sampling distribution
    '''
    n = len(X)  # Total number of data points
    n_important = np.int(np.floor(frac * n))  # Number of important data points to be selected

    # Split the total number of landmarks (m) into two parts: n0 and n1
    n0, n1 = np.int(np.floor(m / 2)), np.int(np.ceil(m / 2))

    # Step 1: Select n0 landmarks uniformly at random from the data set X
    ind0 = np.random.choice(n, n0, replace=False)
    Z1 = X[ind0]  # Z1 represents the uniformly selected landmarks

    # Step 2: Use importance sampling to select n1 landmarks based on a distribution proportional to importance scores
    ind_remain = np.delete(np.arange(n), ind0)
    X_sub = X[ind_remain]  # Subsampled data set without the previously selected landmarks
    Dist = euclidean_distances(X_sub, Z1, squared=False) / sigma  # Compute distances between points and landmarks
    DistMin = Dist.min(axis=1)  # Minimum distance to any landmark for each point
    Prob = (1 - omega) / len(ind_remain) + omega * (DistMin / np.sum(DistMin))  # Importance scores for data points
    ind1 = np.random.choice(len(X_sub), size=n_important, replace=False, p=Prob)  # Sample n1 points based on scores
    Z2 = KMeans(n_clusters=n1, max_iter=100).fit(X_sub[ind1]).cluster_centers_  # Apply K-means to get n1 cluster centroids

    # Concatenate the two sets of landmarks to get the final set
    Z = np.concatenate((Z1, Z2), axis=0)

    return Z  # Return the final set of importance-sampled landmarks

def nystrom(X, Z, sigma, r):
    '''Computing low-rank approximation using the Nystrom method'''
    # Compute the Nystrom low-rank approximation of the kernel matrix for given data points X and landmarks Z
    C = np.exp(-euclidean_distances(X, Z, squared=True) / (sigma ** 2))  # Kernel matrix between X and Z
    W = np.exp(-euclidean_distances(Z, squared=True) / (sigma ** 2))  # Kernel matrix among the landmarks Z
    W = (W + W.T) / 2  # Symmetrize the kernel matrix W
    Q, R = la.qr(C, mode='reduced')  # Compute the reduced QR decomposition of the kernel matrix C
    V, Sigma, _ = la.svd(la.multi_dot([R, np.linalg.pinv(W), np.transpose(R)]))  # Compute SVD for Nystrom approximation
    EigVecNys = np.matmul(Q, V[:, :r])  # Nystrom low-rank eigenvectors
    EigValNys = Sigma[:r]  # Nystrom low-rank eigenvalues
    return EigVecNys, EigValNys  # Return the Nystrom low-rank approximation
